In [1]:
# This project is a proof of concept for an application that selects the closest match between neighborhoods for someone interested in relocation
# For this proof of concept, the current city (Toronto) and neighborhood (Studio District) is hardocded along with the city to which he/she is moving (Manhattan)

In [2]:
# Get geo data for current location (city) using downloaded file "Toronto Neighborhoods.csv"

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_29772880583a4ae1bc7b6bdd4bf0883a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='GWuIrAPLvTZ9rIR2tWkxpDTzQXzulM1ljoTBiu5dyykH',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_29772880583a4ae1bc7b6bdd4bf0883a.get_object(Bucket='ibmdatasciencecourseracapstonepro-donotdelete-pr-vajqwaghuh8zp4',Key='Toronto Neighborhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_curr_city = pd.read_csv(body)
df_curr_city.head()



,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [3]:
# Set current neighborhood ("curr_hood_geo") to the "Studio District" in Toronto using the index

curr_hood_geo = df_curr_city.iloc[[3]] # this can be changed in the code to sample different neighborhoods
curr_lat = curr_hood_geo["Latitude"]
curr_long = curr_hood_geo["Longitude"]

curr_hood_geo

,Postcode,Borough,Neighborhood,Latitude,Longitude
3,M4M,East Toronto,Studio District,43.659526,-79.340923


In [4]:
# Importing necessary packages

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [5]:
# Foursquare Credentials; Confidiential Info for Foursquare API removed

CLIENT_ID = 'QQ0PGTCKV4DTF33EGB404UJE1LK32KHR431VJ2XGBHY2YUDE' # your Foursquare ID
CLIENT_SECRET = 'XBRTKVEH14PBAQNRBHYEQNBAJOWFYIEOKPOYLZSU5HIWXJB4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [6]:
curr_hood_geo

,Postcode,Borough,Neighborhood,Latitude,Longitude
3,M4M,East Toronto,Studio District,43.659526,-79.340923


In [7]:
# Create a venue profile for the current neighborhood using the top 50 venues for the current neighborhood area within a radius of 500 meters of that neighborhood area

LIMIT = 50 # limit of number of venues returned by Foursquare API

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

curr_hood_venues = getNearbyVenues(names=curr_hood_geo['Neighborhood'],
                                   latitudes=curr_hood_geo['Latitude'],
                                   longitudes=curr_hood_geo['Longitude']
                                  )


# Normalizing and analyzing each neighborhood...


# one hot encoding
curr_hood_onehot = pd.get_dummies(curr_hood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
curr_hood_onehot['Neighborhood'] = curr_hood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [curr_hood_onehot.columns[-1]] + list(curr_hood_onehot.columns[:-1])
curr_hood_onehot = curr_hood_onehot[fixed_columns]

curr_hood_onehot.head()


# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


curr_hood_grouped = curr_hood_onehot.groupby('Neighborhood').mean().reset_index()
curr_hood_grouped

Studio District


,Neighborhood,Yoga Studio,American Restaurant,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,...,Latin American Restaurant,Middle Eastern Restaurant,Park,Pet Store,Sandwich Place,Seafood Restaurant,Stationery Store,Thai Restaurant,Thrift / Vintage Store,Wine Bar
0,Studio District,0.02381,0.047619,0.047619,0.02381,0.02381,0.02381,0.047619,0.095238,0.02381,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381


In [8]:
# Get geo data for target location (city) using downloaded file "Manhattan Neighborhoods.csv"

body2 = client_29772880583a4ae1bc7b6bdd4bf0883a.get_object(Bucket='ibmdatasciencecourseracapstonepro-donotdelete-pr-vajqwaghuh8zp4',Key='Manhattan Neighborhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body2, "__iter__"): body2.__iter__ = types.MethodType( __iter__, body2 )

df_tgt_city = pd.read_csv(body2)
df_tgt_city.head()


,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [9]:
tgt_hood_geo = df_tgt_city

# Create a venue profiles for the target neighborhoods using the top 50 venues for the target city neighborhoods area within a radius of 500 meters of each neighborhood area

# LIMIT = 50 # limit of number of venues returned by Foursquare API 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

tgt_hood_venues = getNearbyVenues(names=tgt_hood_geo['Neighborhood'],
                                   latitudes=tgt_hood_geo['Latitude'],
                                   longitudes=tgt_hood_geo['Longitude']
                                  )


# Normalizing and analyzing each neighborhood...


# one hot encoding
tgt_hood_onehot = pd.get_dummies(tgt_hood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tgt_hood_onehot['Neighborhood'] = tgt_hood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tgt_hood_onehot.columns[-1]] + list(tgt_hood_onehot.columns[:-1])
tgt_hood_onehot = tgt_hood_onehot[fixed_columns]

tgt_hood_onehot.head()


# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


tgt_hood_grouped = tgt_hood_onehot.groupby('Neighborhood').mean().reset_index()
tgt_hood_grouped

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.00,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.02,0.000000
1,Carnegie Hill,0.00,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.02,0.04,0.00,0.00,0.040000
2,Central Harlem,0.00,0.00,0.00,0.066667,0.044444,0.00,0.00,0.00,0.022222,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000
3,Chelsea,0.00,0.00,0.00,0.000000,0.040000,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.02,0.00,0.02,0.000000
4,Chinatown,0.00,0.00,0.00,0.000000,0.040000,0.00,0.00,0.00,0.000000,...,0.00,0.02,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.020000
5,Civic Center,0.00,0.00,0.00,0.000000,0.040000,0.02,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.02,0.00,0.00,0.020000
6,Clinton,0.00,0.00,0.00,0.000000,0.060000,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.02,0.04,0.00,0.00,0.000000
7,East Harlem,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000
8,East Village,0.00,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.000000,...,0.00,0.04,0.00,0.000000,0.00,0.04,0.02,0.00,0.00,0.000000
9,Financial District,0.02,0.00,0.00,0.000000,0.040000,0.00,0.00,0.00,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.04,0.00,0.00,0.000000


In [10]:
# Transpose the current neighborhood profile in preparation to join with the target neighborhoods

test_curr_x=curr_hood_grouped.transpose()
test_curr_x.reset_index(inplace=True)
test_curr_x

,index,0
0,Neighborhood,Studio District
1,Yoga Studio,0.0238095
2,American Restaurant,0.047619
3,Bakery,0.047619
4,Bank,0.0238095
5,Bar,0.0238095
6,Bookstore,0.0238095
7,Brewery,0.047619
8,Café,0.0952381
9,Cheese Shop,0.0238095


In [11]:
# Transpose the target neighborhood profiles in preparation to join with the current neighborhood

test_tgt_x=tgt_hood_grouped.transpose()
test_tgt_x.reset_index(inplace=True)
test_tgt_x

,index,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,Neighborhood,Battery Park City,Carnegie Hill,Central Harlem,Chelsea,Chinatown,Civic Center,Clinton,East Harlem,East Village,...,Stuyvesant Town,Sutton Place,Tribeca,Tudor City,Turtle Bay,Upper East Side,Upper West Side,Washington Heights,West Village,Yorkville
1,Accessories Store,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.02,0.02,0.02,0
2,Adult Boutique,0,0,0,0,0,0,0,0,0,...,0,0.02,0,0,0,0,0,0,0,0
3,Afghan Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,African Restaurant,0,0,0.0666667,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,American Restaurant,0.02,0.02,0.0444444,0.04,0.04,0.04,0.06,0,0.02,...,0,0.02,0.06,0.02,0.02,0.04,0.04,0.02,0.04,0
6,Antique Shop,0,0,0,0,0,0.02,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Arcade,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Argentinian Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0.02,0,0,0,0,0,0,0
9,Art Gallery,0,0,0.0222222,0,0,0,0,0,0,...,0,0,0.02,0,0,0.02,0,0,0,0


In [12]:
# Merge the current and target neighborhood profiles based only on venues matching the current neighborhood profile

test_df=pd.merge(test_curr_x,test_tgt_x, on="index", how="left")
test_df

,index,0_x,0_y,1,2,3,4,5,6,7,...,30,31,32,33,34,35,36,37,38,39
0,Neighborhood,Studio District,Battery Park City,Carnegie Hill,Central Harlem,Chelsea,Chinatown,Civic Center,Clinton,East Harlem,...,Stuyvesant Town,Sutton Place,Tribeca,Tudor City,Turtle Bay,Upper East Side,Upper West Side,Washington Heights,West Village,Yorkville
1,Yoga Studio,0.0238095,0,0.04,0,0,0.02,0.02,0,0,...,0,0.04,0.02,0.02,0,0.02,0.02,0,0,0
2,American Restaurant,0.047619,0.02,0.02,0.0444444,0.04,0.04,0.04,0.06,0,...,0,0.02,0.06,0.02,0.02,0.04,0.04,0.02,0.04,0
3,Bakery,0.047619,0,0.02,0,0.04,0.02,0.04,0,0.0930233,...,0,0.02,0.02,0,0,0.04,0.04,0.04,0.04,0.02
4,Bank,0.0238095,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.02
5,Bar,0.0238095,0,0.02,0.0444444,0.02,0,0.02,0,0,...,0.111111,0,0,0.02,0,0.02,0.04,0.02,0,0.02
6,Bookstore,0.0238095,0,0.06,0.0222222,0,0,0.02,0,0,...,0,0,0,0,0.02,0.02,0.02,0,0,0.02
7,Brewery,0.047619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Café,0.0952381,0,0.06,0.0222222,0.02,0,0,0.02,0.0232558,...,0,0,0.06,0.02,0.02,0,0.02,0.06,0.02,0.02
9,Cheese Shop,0.0238095,0,0,0,0,0,0,0,0,...,0,0,0,0,0.02,0,0,0,0,0


In [13]:
# Reformat (transpose) the combined neighborhood profiles by neighborhood

test_df_x=test_df.transpose()
#test_tgt_x.reset_index(inplace=True)
test_df_x

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
index,Neighborhood,Yoga Studio,American Restaurant,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,...,Latin American Restaurant,Middle Eastern Restaurant,Park,Pet Store,Sandwich Place,Seafood Restaurant,Stationery Store,Thai Restaurant,Thrift / Vintage Store,Wine Bar
0_x,Studio District,0.0238095,0.047619,0.047619,0.0238095,0.0238095,0.0238095,0.047619,0.0952381,0.0238095,...,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095
0_y,Battery Park City,0,0.02,0,0,0,0,NaN,0,0,...,0,0,0.08,0,0.04,0,NaN,0,0,0
1,Carnegie Hill,0.04,0.02,0.02,0,0.02,0.06,NaN,0.06,0,...,0,0,0,0.02,0,0,NaN,0,0,0.02
2,Central Harlem,0,0.0444444,0,0,0.0444444,0.0222222,NaN,0.0222222,0,...,0,0,0.0222222,0,0,0.0444444,NaN,0,0,0
3,Chelsea,0,0.04,0.04,0,0.02,0,NaN,0.02,0,...,0,0,0,0.02,0.02,0.04,NaN,0,0,0
4,Chinatown,0.02,0.04,0.02,0,0,0,NaN,0,0,...,0,0,0,0,0.04,0,NaN,0.02,0,0
5,Civic Center,0.02,0.04,0.04,0,0.02,0.02,NaN,0,0,...,0,0,0.04,0,0.04,0,NaN,0,0,0
6,Clinton,0,0.06,0,0,0,0,NaN,0.02,0,...,0,0,0.02,0,0.02,0.02,NaN,0,0,0.02
7,East Harlem,0,0,0.0930233,0,0,0,NaN,0.0232558,0,...,0.0697674,0,0.0232558,0,0.0232558,0.0232558,NaN,0.0697674,0,0


In [14]:
# Replace NaN with "0's" in preparation for analysis

test_df_x=test_df_x.fillna(0)
test_df_x

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
index,Neighborhood,Yoga Studio,American Restaurant,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,...,Latin American Restaurant,Middle Eastern Restaurant,Park,Pet Store,Sandwich Place,Seafood Restaurant,Stationery Store,Thai Restaurant,Thrift / Vintage Store,Wine Bar
0_x,Studio District,0.0238095,0.047619,0.047619,0.0238095,0.0238095,0.0238095,0.047619,0.0952381,0.0238095,...,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095
0_y,Battery Park City,0,0.02,0,0,0,0,0,0,0,...,0,0,0.08,0,0.04,0,0,0,0,0
1,Carnegie Hill,0.04,0.02,0.02,0,0.02,0.06,0,0.06,0,...,0,0,0,0.02,0,0,0,0,0,0.02
2,Central Harlem,0,0.0444444,0,0,0.0444444,0.0222222,0,0.0222222,0,...,0,0,0.0222222,0,0,0.0444444,0,0,0,0
3,Chelsea,0,0.04,0.04,0,0.02,0,0,0.02,0,...,0,0,0,0.02,0.02,0.04,0,0,0,0
4,Chinatown,0.02,0.04,0.02,0,0,0,0,0,0,...,0,0,0,0,0.04,0,0,0.02,0,0
5,Civic Center,0.02,0.04,0.04,0,0.02,0.02,0,0,0,...,0,0,0.04,0,0.04,0,0,0,0,0
6,Clinton,0,0.06,0,0,0,0,0,0.02,0,...,0,0,0.02,0,0.02,0.02,0,0,0,0.02
7,East Harlem,0,0,0.0930233,0,0,0,0,0.0232558,0,...,0.0697674,0,0.0232558,0,0.0232558,0.0232558,0,0.0697674,0,0


In [15]:
test_df_x.rename(columns=test_df_x.iloc[0],inplace=True)
test_df_x.drop(test_df_x.index[0], inplace=True)
#test_df_x.set_index("Neighborhood", inplace=True)
test_df_x

##test_df_x.rename(columns=test_df_x.iloc[0]).drop(test_df_x.index[0])
##test_df_x.head()

,Neighborhood,Yoga Studio,American Restaurant,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,...,Latin American Restaurant,Middle Eastern Restaurant,Park,Pet Store,Sandwich Place,Seafood Restaurant,Stationery Store,Thai Restaurant,Thrift / Vintage Store,Wine Bar
0_x,Studio District,0.0238095,0.047619,0.047619,0.0238095,0.0238095,0.0238095,0.047619,0.0952381,0.0238095,...,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095,0.0238095
0_y,Battery Park City,0,0.02,0,0,0,0,0,0,0,...,0,0,0.08,0,0.04,0,0,0,0,0
1,Carnegie Hill,0.04,0.02,0.02,0,0.02,0.06,0,0.06,0,...,0,0,0,0.02,0,0,0,0,0,0.02
2,Central Harlem,0,0.0444444,0,0,0.0444444,0.0222222,0,0.0222222,0,...,0,0,0.0222222,0,0,0.0444444,0,0,0,0
3,Chelsea,0,0.04,0.04,0,0.02,0,0,0.02,0,...,0,0,0,0.02,0.02,0.04,0,0,0,0
4,Chinatown,0.02,0.04,0.02,0,0,0,0,0,0,...,0,0,0,0,0.04,0,0,0.02,0,0
5,Civic Center,0.02,0.04,0.04,0,0.02,0.02,0,0,0,...,0,0,0.04,0,0.04,0,0,0,0,0
6,Clinton,0,0.06,0,0,0,0,0,0.02,0,...,0,0,0.02,0,0.02,0.02,0,0,0,0.02
7,East Harlem,0,0,0.0930233,0,0,0,0,0.0232558,0,...,0.0697674,0,0.0232558,0,0.0232558,0.0232558,0,0.0697674,0,0
8,East Village,0,0.02,0.04,0,0.06,0,0,0,0.02,...,0,0.02,0.02,0,0,0,0,0,0,0.04


In [16]:
# Remove current neighborhood from combined neighborhood dataframe for analytic processing

tgt_hood_work=test_df_x.drop(test_df_x.index[0])
tgt_hood_work.set_index("Neighborhood",inplace=True)
tgt_hood_work

,Yoga Studio,American Restaurant,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,Clothing Store,...,Latin American Restaurant,Middle Eastern Restaurant,Park,Pet Store,Sandwich Place,Seafood Restaurant,Stationery Store,Thai Restaurant,Thrift / Vintage Store,Wine Bar
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Battery Park City,0,0.02,0,0,0,0,0,0,0,0,...,0,0,0.08,0,0.04,0,0,0,0,0
Carnegie Hill,0.04,0.02,0.02,0,0.02,0.06,0,0.06,0,0,...,0,0,0,0.02,0,0,0,0,0,0.02
Central Harlem,0,0.0444444,0,0,0.0444444,0.0222222,0,0.0222222,0,0,...,0,0,0.0222222,0,0,0.0444444,0,0,0,0
Chelsea,0,0.04,0.04,0,0.02,0,0,0.02,0,0,...,0,0,0,0.02,0.02,0.04,0,0,0,0
Chinatown,0.02,0.04,0.02,0,0,0,0,0,0,0,...,0,0,0,0,0.04,0,0,0.02,0,0
Civic Center,0.02,0.04,0.04,0,0.02,0.02,0,0,0,0,...,0,0,0.04,0,0.04,0,0,0,0,0
Clinton,0,0.06,0,0,0,0,0,0.02,0,0,...,0,0,0.02,0,0.02,0.02,0,0,0,0.02
East Harlem,0,0,0.0930233,0,0,0,0,0.0232558,0,0,...,0.0697674,0,0.0232558,0,0.0232558,0.0232558,0,0.0697674,0,0
East Village,0,0.02,0.04,0,0.06,0,0,0,0.02,0,...,0,0.02,0.02,0,0,0,0,0,0,0.04


In [17]:
curr_hood_grouped.set_index("Neighborhood", inplace=True)
curr_hood_grouped

,Yoga Studio,American Restaurant,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,Clothing Store,...,Latin American Restaurant,Middle Eastern Restaurant,Park,Pet Store,Sandwich Place,Seafood Restaurant,Stationery Store,Thai Restaurant,Thrift / Vintage Store,Wine Bar
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Studio District,0.02381,0.047619,0.047619,0.02381,0.02381,0.02381,0.047619,0.095238,0.02381,0.02381,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381


In [18]:
curr_hood_grouped.columns

Index(['Yoga Studio', 'American Restaurant', 'Bakery', 'Bank', 'Bar',
       'Bookstore', 'Brewery', 'Café', 'Cheese Shop', 'Clothing Store',
       'Coffee Shop', 'Comfort Food Restaurant', 'Convenience Store',
       'Coworking Space', 'Department Store', 'Diner', 'Fish Market',
       'Gastropub', 'Gay Bar', 'Gym / Fitness Center', 'Ice Cream Shop',
       'Italian Restaurant', 'Latin American Restaurant',
       'Middle Eastern Restaurant', 'Park', 'Pet Store', 'Sandwich Place',
       'Seafood Restaurant', 'Stationery Store', 'Thai Restaurant',
       'Thrift / Vintage Store', 'Wine Bar'],
      dtype='object')

In [19]:
# Calculate the similarity between the current neighborhood and the target city neighborhoods using Euclidean distance

import math
from pandas import DataFrame

def euclidean_distance(row):    # Simple Euclidean Distance Function
    inner_value = 0
    for k in curr_hood_grouped.columns:
        inner_value += (row[k] - curr_hood_grouped[k]) ** 2
    return math.sqrt(inner_value)

# Find the Euclidean distance from each neighborhood venue profile in the target city to the venue profile in the current city.
hood_distance = tgt_hood_work.apply(euclidean_distance, axis=1)
match_df=DataFrame(hood_distance, columns=['Match_Distance'])
match_df.sort_values(['Match_Distance'],ascending=True, axis=0, inplace=True)
match_df

,Match_Distance
Neighborhood,
Hamilton Heights,0.128363
Lower East Side,0.130570
Washington Heights,0.133241
Carnegie Hill,0.135790
Inwood,0.139116
Flatiron,0.142498
Chelsea,0.144950
Tribeca,0.146323
Little Italy,0.150178


In [20]:
tgt_hood_work

,Yoga Studio,American Restaurant,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,Clothing Store,...,Latin American Restaurant,Middle Eastern Restaurant,Park,Pet Store,Sandwich Place,Seafood Restaurant,Stationery Store,Thai Restaurant,Thrift / Vintage Store,Wine Bar
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Battery Park City,0,0.02,0,0,0,0,0,0,0,0,...,0,0,0.08,0,0.04,0,0,0,0,0
Carnegie Hill,0.04,0.02,0.02,0,0.02,0.06,0,0.06,0,0,...,0,0,0,0.02,0,0,0,0,0,0.02
Central Harlem,0,0.0444444,0,0,0.0444444,0.0222222,0,0.0222222,0,0,...,0,0,0.0222222,0,0,0.0444444,0,0,0,0
Chelsea,0,0.04,0.04,0,0.02,0,0,0.02,0,0,...,0,0,0,0.02,0.02,0.04,0,0,0,0
Chinatown,0.02,0.04,0.02,0,0,0,0,0,0,0,...,0,0,0,0,0.04,0,0,0.02,0,0
Civic Center,0.02,0.04,0.04,0,0.02,0.02,0,0,0,0,...,0,0,0.04,0,0.04,0,0,0,0,0
Clinton,0,0.06,0,0,0,0,0,0.02,0,0,...,0,0,0.02,0,0.02,0.02,0,0,0,0.02
East Harlem,0,0,0.0930233,0,0,0,0,0.0232558,0,0,...,0.0697674,0,0.0232558,0,0.0232558,0.0232558,0,0.0697674,0,0
East Village,0,0.02,0.04,0,0.06,0,0,0,0.02,0,...,0,0.02,0.02,0,0,0,0,0,0,0.04


In [21]:
final_df=tgt_hood_work.append(curr_hood_grouped)
final_df

,Yoga Studio,American Restaurant,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,Clothing Store,...,Latin American Restaurant,Middle Eastern Restaurant,Park,Pet Store,Sandwich Place,Seafood Restaurant,Stationery Store,Thai Restaurant,Thrift / Vintage Store,Wine Bar
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Battery Park City,0,0.02,0,0,0,0,0,0,0,0,...,0,0,0.08,0,0.04,0,0,0,0,0
Carnegie Hill,0.04,0.02,0.02,0,0.02,0.06,0,0.06,0,0,...,0,0,0,0.02,0,0,0,0,0,0.02
Central Harlem,0,0.0444444,0,0,0.0444444,0.0222222,0,0.0222222,0,0,...,0,0,0.0222222,0,0,0.0444444,0,0,0,0
Chelsea,0,0.04,0.04,0,0.02,0,0,0.02,0,0,...,0,0,0,0.02,0.02,0.04,0,0,0,0
Chinatown,0.02,0.04,0.02,0,0,0,0,0,0,0,...,0,0,0,0,0.04,0,0,0.02,0,0
Civic Center,0.02,0.04,0.04,0,0.02,0.02,0,0,0,0,...,0,0,0.04,0,0.04,0,0,0,0,0
Clinton,0,0.06,0,0,0,0,0,0.02,0,0,...,0,0,0.02,0,0.02,0.02,0,0,0,0.02
East Harlem,0,0,0.0930233,0,0,0,0,0.0232558,0,0,...,0.0697674,0,0.0232558,0,0.0232558,0.0232558,0,0.0697674,0,0
East Village,0,0.02,0.04,0,0.06,0,0,0,0.02,0,...,0,0.02,0.02,0,0,0,0,0,0,0.04


In [22]:
# add distance to the df

final_df=pd.merge(final_df,match_df, on="Neighborhood", how="left")
final_df

,Yoga Studio,American Restaurant,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,Clothing Store,...,Middle Eastern Restaurant,Park,Pet Store,Sandwich Place,Seafood Restaurant,Stationery Store,Thai Restaurant,Thrift / Vintage Store,Wine Bar,Match_Distance
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Battery Park City,0,0.02,0,0,0,0,0,0,0,0,...,0,0.08,0,0.04,0,0,0,0,0,0.176422
Carnegie Hill,0.04,0.02,0.02,0,0.02,0.06,0,0.06,0,0,...,0,0,0.02,0,0,0,0,0,0.02,0.135790
Central Harlem,0,0.0444444,0,0,0.0444444,0.0222222,0,0.0222222,0,0,...,0,0.0222222,0,0,0.0444444,0,0,0,0,0.173234
Chelsea,0,0.04,0.04,0,0.02,0,0,0.02,0,0,...,0,0,0.02,0.02,0.04,0,0,0,0,0.144950
Chinatown,0.02,0.04,0.02,0,0,0,0,0,0,0,...,0,0,0,0.04,0,0,0.02,0,0,0.176206
Civic Center,0.02,0.04,0.04,0,0.02,0.02,0,0,0,0,...,0,0.04,0,0.04,0,0,0,0,0,0.159884
Clinton,0,0.06,0,0,0,0,0,0.02,0,0,...,0,0.02,0,0.02,0.02,0,0,0,0.02,0.172712
East Harlem,0,0,0.0930233,0,0,0,0,0.0232558,0,0,...,0,0.0232558,0,0.0232558,0.0232558,0,0.0697674,0,0,0.186309
East Village,0,0.02,0.04,0,0.06,0,0,0,0.02,0,...,0.02,0.02,0,0,0,0,0,0,0.04,0.174031


In [23]:
# Replace NaN in Current Neighborhood Row, then reorder based on Euclidean Distance

final_df=final_df.fillna(0)
final_df.sort_values('Match_Distance',ascending=True,axis=0,inplace=True)
final_df

,Yoga Studio,American Restaurant,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,Clothing Store,...,Middle Eastern Restaurant,Park,Pet Store,Sandwich Place,Seafood Restaurant,Stationery Store,Thai Restaurant,Thrift / Vintage Store,Wine Bar,Match_Distance
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Studio District,0.023810,0.047619,0.047619,0.023810,0.023810,0.023810,0.047619,0.095238,0.02381,0.02381,...,0.02381,0.023810,0.02381,0.023810,0.023810,0.02381,0.023810,0.02381,0.02381,0.000000
Hamilton Heights,0.040000,0.000000,0.040000,0.000000,0.020000,0.000000,0.000000,0.080000,0.00000,0.00000,...,0.00000,0.040000,0.00000,0.020000,0.020000,0.00000,0.000000,0.00000,0.02000,0.128363
Lower East Side,0.020000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.060000,0.00000,0.02000,...,0.00000,0.020000,0.00000,0.000000,0.000000,0.00000,0.020000,0.00000,0.00000,0.130570
Washington Heights,0.000000,0.020000,0.040000,0.000000,0.020000,0.000000,0.000000,0.060000,0.00000,0.00000,...,0.00000,0.040000,0.00000,0.000000,0.020000,0.00000,0.000000,0.00000,0.00000,0.133241
Carnegie Hill,0.040000,0.020000,0.020000,0.000000,0.020000,0.060000,0.000000,0.060000,0.00000,0.00000,...,0.00000,0.000000,0.02000,0.000000,0.000000,0.00000,0.000000,0.00000,0.02000,0.135790
Inwood,0.020000,0.040000,0.040000,0.000000,0.020000,0.000000,0.000000,0.060000,0.00000,0.00000,...,0.00000,0.040000,0.02000,0.000000,0.020000,0.00000,0.000000,0.00000,0.04000,0.139116
Flatiron,0.040000,0.040000,0.020000,0.000000,0.000000,0.020000,0.000000,0.040000,0.02000,0.02000,...,0.00000,0.000000,0.00000,0.020000,0.000000,0.00000,0.020000,0.00000,0.00000,0.142498
Chelsea,0.000000,0.040000,0.040000,0.000000,0.020000,0.000000,0.000000,0.020000,0.00000,0.00000,...,0.00000,0.000000,0.02000,0.020000,0.040000,0.00000,0.000000,0.00000,0.00000,0.144950
Tribeca,0.020000,0.060000,0.020000,0.000000,0.000000,0.000000,0.000000,0.060000,0.00000,0.00000,...,0.00000,0.040000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.04000,0.146323


In [24]:
# RESULTS

print('\033[1m'+"The closest match to your current neighborhood is "+ final_df.index.values[1]+'\033[0m',"\n")

print(final_df.iloc[[0,1]])


The closest match to your current neighborhood is Hamilton Heights 

                  Yoga Studio  American Restaurant    Bakery     Bank  \
Neighborhood                                                            
Studio District       0.02381             0.047619  0.047619  0.02381   
Hamilton Heights      0.04000             0.000000  0.040000  0.00000   

                      Bar  Bookstore   Brewery      Café  Cheese Shop  \
Neighborhood                                                            
Studio District   0.02381    0.02381  0.047619  0.095238      0.02381   
Hamilton Heights  0.02000    0.00000  0.000000  0.080000      0.00000   

                  Clothing Store  ...  Middle Eastern Restaurant     Park  \
Neighborhood                      ...                                       
Studio District          0.02381  ...                    0.02381  0.02381   
Hamilton Heights         0.00000  ...                    0.00000  0.04000   

                  Pet Store  Sandwic

In [25]:
#curr_hood_geo[["Neighborhood","Latitude","Longitude"]]
curr_lat=curr_hood_geo["Latitude"]
curr_lat
curr_lon=curr_hood_geo["Longitude"]
curr_lon



3   -79.340923
Name: Longitude, dtype: float64

In [26]:
curr_hood_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Studio District,43.659526,-79.340923,Ed's Real Scoop,43.660656,-79.342019,Ice Cream Shop
1,Studio District,43.659526,-79.340923,Leslieville Pumps,43.660892,-79.340626,Sandwich Place
2,Studio District,43.659526,-79.340923,Queen Books,43.660651,-79.342267,Bookstore
3,Studio District,43.659526,-79.340923,Te Aro,43.661373,-79.338577,Coffee Shop
4,Studio District,43.659526,-79.340923,The Bone House,43.660894,-79.341097,Pet Store
5,Studio District,43.659526,-79.340923,Hooked,43.660407,-79.343257,Fish Market
6,Studio District,43.659526,-79.340923,Purple Penguin Cafe,43.660501,-79.342565,Café
7,Studio District,43.659526,-79.340923,Mercury Espresso Bar,43.660806,-79.341241,Coffee Shop
8,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood
9,Studio District,43.659526,-79.340923,WAYLABAR,43.661234,-79.339597,Gay Bar


In [27]:
final_hood_venues=tgt_hood_venues.loc[tgt_hood_venues["Neighborhood"]==final_df.index.values[1]]
final_hood_venues


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
174,Hamilton Heights,40.823604,-73.949688,The Grange Bar & Eatery,40.822554,-73.949532,Cocktail Bar
175,Hamilton Heights,40.823604,-73.949688,R.O.K.C,40.823654,-73.952506,Cocktail Bar
176,Hamilton Heights,40.823604,-73.949688,Cafe One,40.822139,-73.949853,Café
177,Hamilton Heights,40.823604,-73.949688,Fumo,40.821412,-73.950499,Italian Restaurant
178,Hamilton Heights,40.823604,-73.949688,Bikram Yoga Harlem,40.825672,-73.948887,Yoga Studio
179,Hamilton Heights,40.823604,-73.949688,Anchor Wine Bar,40.825424,-73.951204,Wine Bar
180,Hamilton Heights,40.823604,-73.949688,Oso,40.821945,-73.950038,Mexican Restaurant
181,Hamilton Heights,40.823604,-73.949688,Wat’s On Your Plate,40.825632,-73.948494,Caribbean Restaurant
182,Hamilton Heights,40.823604,-73.949688,Uncle Tony's Pizza,40.821440,-73.950766,Pizza Place
183,Hamilton Heights,40.823604,-73.949688,Brahman Yoga Studio,40.825319,-73.947848,Yoga Studio


In [28]:
curr_hood_venues.rename(columns={"Venue Latitude": "lat", "Venue Longitude": "lon", "Venue Category": "categories"}, inplace=True)
curr_hood_venues


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,lat,lon,categories
0,Studio District,43.659526,-79.340923,Ed's Real Scoop,43.660656,-79.342019,Ice Cream Shop
1,Studio District,43.659526,-79.340923,Leslieville Pumps,43.660892,-79.340626,Sandwich Place
2,Studio District,43.659526,-79.340923,Queen Books,43.660651,-79.342267,Bookstore
3,Studio District,43.659526,-79.340923,Te Aro,43.661373,-79.338577,Coffee Shop
4,Studio District,43.659526,-79.340923,The Bone House,43.660894,-79.341097,Pet Store
5,Studio District,43.659526,-79.340923,Hooked,43.660407,-79.343257,Fish Market
6,Studio District,43.659526,-79.340923,Purple Penguin Cafe,43.660501,-79.342565,Café
7,Studio District,43.659526,-79.340923,Mercury Espresso Bar,43.660806,-79.341241,Coffee Shop
8,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood
9,Studio District,43.659526,-79.340923,WAYLABAR,43.661234,-79.339597,Gay Bar


In [29]:
#!conda install -c conda-forge folium=0.5.0 --yes 
#import folium # map rendering library

hood1=curr_hood_geo.Neighborhood.values[0]
lat1=curr_hood_geo.Latitude.values[0]
lon1=curr_hood_geo.Longitude.values[0]
venues_map1 = folium.Map(location=[lat1, lon1], zoom_start=15) # generate map centred around Current Neighborhood


# add Current Neighborhood as a red circle mark
folium.features.CircleMarker(
    [lat1, lon1],
    radius=10,
    popup=hood1,
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map1)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(curr_hood_venues.lat, curr_hood_venues.lon, curr_hood_venues.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map1)

# display map
venues_map1

In [30]:
final_hood_venues.rename(columns={"Venue Latitude": "lat", "Venue Longitude": "lon", "Venue Category": "categories"}, inplace=True)
final_hood_venues

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,lat,lon,categories
174,Hamilton Heights,40.823604,-73.949688,The Grange Bar & Eatery,40.822554,-73.949532,Cocktail Bar
175,Hamilton Heights,40.823604,-73.949688,R.O.K.C,40.823654,-73.952506,Cocktail Bar
176,Hamilton Heights,40.823604,-73.949688,Cafe One,40.822139,-73.949853,Café
177,Hamilton Heights,40.823604,-73.949688,Fumo,40.821412,-73.950499,Italian Restaurant
178,Hamilton Heights,40.823604,-73.949688,Bikram Yoga Harlem,40.825672,-73.948887,Yoga Studio
179,Hamilton Heights,40.823604,-73.949688,Anchor Wine Bar,40.825424,-73.951204,Wine Bar
180,Hamilton Heights,40.823604,-73.949688,Oso,40.821945,-73.950038,Mexican Restaurant
181,Hamilton Heights,40.823604,-73.949688,Wat’s On Your Plate,40.825632,-73.948494,Caribbean Restaurant
182,Hamilton Heights,40.823604,-73.949688,Uncle Tony's Pizza,40.821440,-73.950766,Pizza Place
183,Hamilton Heights,40.823604,-73.949688,Brahman Yoga Studio,40.825319,-73.947848,Yoga Studio


In [31]:
#need to set lat-lon for final hood


hood2=final_df.index.values[1]
lat2=final_hood_venues["Neighborhood Latitude"].values[0]
lon2=final_hood_venues["Neighborhood Longitude"].values[0]

In [32]:
venues_map2 = folium.Map(location=[lat2, lon2], zoom_start=15) # generate map centred around Current Neighborhood


# add Current Neighborhood as a red circle mark
folium.features.CircleMarker(
    [lat2, lon2],
    radius=10,
    popup=hood2,
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map2)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(final_hood_venues.lat, final_hood_venues.lon, final_hood_venues.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map2)

# display map
venues_map2

In [35]:
final_match=final_df.iloc[0:2]
final_match

,Yoga Studio,American Restaurant,Bakery,Bank,Bar,Bookstore,Brewery,Café,Cheese Shop,Clothing Store,...,Middle Eastern Restaurant,Park,Pet Store,Sandwich Place,Seafood Restaurant,Stationery Store,Thai Restaurant,Thrift / Vintage Store,Wine Bar,Match_Distance
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Studio District,0.02381,0.047619,0.047619,0.02381,0.02381,0.02381,0.047619,0.095238,0.02381,0.02381,...,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.02381,0.000000
Hamilton Heights,0.04000,0.000000,0.040000,0.00000,0.02000,0.00000,0.000000,0.080000,0.00000,0.00000,...,0.00000,0.04000,0.00000,0.02000,0.02000,0.00000,0.00000,0.00000,0.02000,0.128363


In [36]:
final_match_x=final_match.transpose()
final_match_x

Neighborhood,Studio District,Hamilton Heights
Yoga Studio,0.023810,0.040000
American Restaurant,0.047619,0.000000
Bakery,0.047619,0.040000
Bank,0.023810,0.000000
Bar,0.023810,0.020000
Bookstore,0.023810,0.000000
Brewery,0.047619,0.000000
Café,0.095238,0.080000
Cheese Shop,0.023810,0.000000
Clothing Store,0.023810,0.000000
